In [1]:
import pandas as pd

# 熱源機の最大能力

# 概要

熱源機の最大能力を求める。  
最大暖房能力は、設置室の床面積に応じて設定する。

$ q_{max,i} = \alpha \times A_{R,i} \times f_{cT} \times f_{cl} $

ここで、  
$q_{max,i}$ :  設置機器の暖房最大能力, W  
$\alpha$ : 連続運転時における単位面積あたりの最大暖房負荷を表す係数, W m<sup>-2</sup>  
$A_{R,i}$ : 標準住宅における居室$i$の床面積, m<sup>2</sup>  
$f_{cT}$ : 外気温度補正係数 (=1.05)  
$f_{cl}$ : 間欠運転能力補正係数  
である。

最大能力は暖房負荷計算結果の最大値とし、それを床面積当たりにして保持する。

係数$\alpha$及び$f_{cl}$は、省エネ基準における各室の居室連続運転および居室間歇運転時の暖房負荷の最大値から求めることとする。

## 使用する暖房負荷の諸条件とファイル名

暖房負荷は居室連続運転および居室間歇運転の場合を用い、断熱性能などの条件は以下とする。
- 断熱水準:H11基準相当
- 日射遮蔽レベル:中
- 蓄熱:なし

該当するCSVファイル名は以下のとおり。

| 地域の区分 | 居室連続運転 | 居室間歇運転 |
| :---: | :---: | :---: |
| 1 | IH1x_13_05_212_1SS.csv | IH1x_12_05_212_1SS.csv |
| 2 | IH2x_13_05_212_1SS.csv | IH2x_12_05_212_1SS.csv |
| 3 | IH3x_13_05_212_1SS.csv | IH3x_12_05_212_1SS.csv |
| 4 | IH4x_13_05_212_1SS.csv | IH4x_12_05_212_1SS.csv |
| 5 | IH5x_13_05_212_1SS.csv | IH5x_12_05_212_1SS.csv |
| 6 | IH6x_13_05_212_1SS.csv | IH6x_12_05_212_1SS.csv |
| 7 | IH7x_13_05_212_1SS.csv | IH7x_12_05_212_1SS.csv |

In [2]:
# 居室連続運転のファイル
filenames_cnt = {
    '1' : 'IH1x_13_05_212_1SS.csv',
    '2' : 'IH2x_13_05_212_1SS.csv',
    '3' : 'IH3x_13_05_212_1SS.csv',
    '4' : 'IH4x_13_05_212_1SS.csv',
    '5' : 'IH5x_13_05_212_1SS.csv',
    '6' : 'IH6x_13_05_212_1SS.csv',
    '7' : 'IH7x_13_05_212_1SS.csv',
}
# 居室間歇運転のファイル
filenames_int = {
    '1' : 'IH1x_12_05_212_1SS.csv',
    '2' : 'IH2x_12_05_212_1SS.csv',
    '3' : 'IH3x_12_05_212_1SS.csv',
    '4' : 'IH4x_12_05_212_1SS.csv',
    '5' : 'IH5x_12_05_212_1SS.csv',
    '6' : 'IH6x_12_05_212_1SS.csv',
    '7' : 'IH7x_12_05_212_1SS.csv',
}

## 使用する暖房負荷ファイルの列

主たる居室はLDKの暖房負荷を使用する。  
その他の居室は、和室・主寝室・子供室1・子供室2であるが、そのうち和室は暖房運転の設定がない（運転時間ゼロ）であるため、主寝室・子供室1・子供室2のみ使用する。

また、外気温度・外気湿度も抽出する。

暖房負荷ファイルにおける列名と列インデックス（ゼロ始まり）は以下のとおり。

| 列名 | 列インデックス |
| --- | --- |
| 外気温度(℃) | 3 |
| 外気湿度(g/kgDA) | 4 |
| LDK | 5 |
| 主寝室 | 9 |
| 子供室1 | 11 |
| 子供室2 | 13 |

In [3]:
columns = {
    '外気温度' : 3,
    '外気湿度' : 4,
    'LDK' : 5,
    '主寝室' : 9,
    '子供室1' : 11,
    '子供室2' : 13,
}

## データの読み込み

In [4]:
# データを読み込む関数
def read_data(fnames):
    # pandas の csv 読み込みを用いる
    # 列ヘッダーは後で付与するのでスキップ(4行分)
    d = pd.read_csv(fnames,
                   encoding='shift_jis',
                   engine='python',
                   header=None,
                   skiprows=4,
                   usecols=columns.values())
    # 列ヘッダーの付与
    d.columns = ['外気温度(℃)','外気湿度(g/kgDA)','LDK','主寝室','子供室1','子供室2']
    return d

datas_cnt = {}
for (region, filename) in filenames_cnt.items():
    datas_cnt[region] = read_data(filename)

datas_int = {}
for (region, filename) in filenames_int.items():
    datas_int[region] = read_data(filename)

### 確認（居室連続）

In [5]:
for (region, d) in datas_cnt.items():
    print('地域:' + str(region))
    display(d.head())

地域:1


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,-13.0,1.3,8452.74,3000.59,2231.82,2594.03
1,-14.2,1.3,8711.91,3104.16,2311.29,2684.75
2,-15.0,1.2,8921.78,3193.17,2373.44,2756.54
3,-16.2,1.0,9187.44,3304.69,2454.76,2848.56
4,-16.7,1.0,9365.67,3382.64,2505.74,2907.98


地域:2


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,-6.9,1.9,7028.55,2453.25,1812.12,2126.02
1,-6.7,1.8,7032.25,2444.80,1806.87,2120.43
2,-6.9,1.9,7068.03,2452.82,1814.05,2127.23
3,-6.9,2.0,7078.04,2452.97,1813.15,2125.51
4,-5.4,2.2,6870.99,2368.19,1739.00,2044.21


地域:3


...

,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,-0.9,2.2,8205.71,2595.16,2111.98,2573.44
1,-2.0,2.1,8641.90,2743.25,2236.25,2714.09
2,-2.8,2.1,9002.92,2873.71,2337.45,2829.83
3,-4.0,2.2,9449.92,3034.02,2466.08,2977.76
4,-4.2,2.3,9681.78,3125.76,2527.47,3049.43


地域:6


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,2.6,3.3,7096.04,2159.22,1760.93,2158.25
1,2.9,3.2,7131.14,2168.41,1762.61,2157.42
2,2.8,3.2,7216.09,2194.25,1782.26,2176.72
3,2.4,3.3,7353.68,2238.87,1819.67,2217.08
4,2.4,3.3,7404.65,2256.97,1830.52,2228.08


地域:7


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,3.5,4.1,5887.27,1741.97,1476.07,1847.72
1,2.9,4.0,6314.29,1899.89,1594.15,1980.17
2,2.7,3.9,6602.52,2011.04,1669.87,2064.79
3,1.8,3.7,6994.51,2152.86,1780.08,2188.67
4,1.3,3.5,7298.36,2266.29,1863.60,2283.26


### 確認（居室間歇）

In [6]:
for (region, d) in datas_int.items():
    print('地域:' + str(region))
    display(d.head())

地域:1


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,-13.0,1.3,0.0,0,0.0,0.0
1,-14.2,1.3,0.0,0,0.0,0.0
2,-15.0,1.2,0.0,0,0.0,0.0
3,-16.2,1.0,0.0,0,0.0,0.0
4,-16.7,1.0,0.0,0,0.0,0.0


地域:2


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,-6.9,1.9,0.0,0,0.0,0.0
1,-6.7,1.8,0.0,0,0.0,0.0
2,-6.9,1.9,0.0,0,0.0,0.0
3,-6.9,2.0,0.0,0,0.0,0.0
4,-5.4,2.2,0.0,0,0.0,0.0


地域:3


...

,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,-0.9,2.2,0.0,0,0.0,0.0
1,-2.0,2.1,0.0,0,0.0,0.0
2,-2.8,2.1,0.0,0,0.0,0.0
3,-4.0,2.2,0.0,0,0.0,0.0
4,-4.2,2.3,0.0,0,0.0,0.0


地域:6


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,2.6,3.3,0.0,0,0.0,0.0
1,2.9,3.2,0.0,0,0.0,0.0
2,2.8,3.2,0.0,0,0.0,0.0
3,2.4,3.3,0.0,0,0.0,0.0
4,2.4,3.3,0.0,0,0.0,0.0


地域:7


,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2
0,3.5,4.1,0.0,0,0.0,0.0
1,2.9,4.0,0.0,0,0.0,0.0
2,2.7,3.9,0.0,0,0.0,0.0
3,1.8,3.7,0.0,0,0.0,0.0
4,1.3,3.5,0.0,0,0.0,0.0


## 主たる居室・その他の居室・全居室の列の追加

In [7]:
def add_rows(d):
    d['主たる居室'] = d['LDK']
    d['その他の居室'] = d['主寝室'] + d['子供室1'] + d['子供室2']
    d['全居室'] = d['主たる居室'] + d['その他の居室']
    
# 居室連続
for d in datas_cnt.values():
    add_rows(d)
# 居室間歇
for d in datas_int.values():
    add_rows(d)

### 確認（居室連続・1地域のみ）

In [8]:
datas_cnt['1'].head()

,外気温度(℃),外気湿度(g/kgDA),LDK,主寝室,子供室1,子供室2,主たる居室,その他の居室,全居室
0,-13.0,1.3,8452.74,3000.59,2231.82,2594.03,8452.74,7826.44,16279.18
1,-14.2,1.3,8711.91,3104.16,2311.29,2684.75,8711.91,8100.20,16812.11
2,-15.0,1.2,8921.78,3193.17,2373.44,2756.54,8921.78,8323.15,17244.93
3,-16.2,1.0,9187.44,3304.69,2454.76,2848.56,9187.44,8608.01,17795.45
4,-16.7,1.0,9365.67,3382.64,2505.74,2907.98,9365.67,8796.36,18162.03


## 最大値を出している行indexの取得とその時の値の取得

In [9]:
def get_id_values(d):
    
    def get_value(target):
        target_id = d[target].idxmax()
        return {
            'index' : int(target_id),
            '外気温度(℃)' : d.iloc[target_id]['外気温度(℃)'],
            '外気湿度(g/kgDA)' : d.iloc[target_id]['外気湿度(g/kgDA)'],
            '負荷(kJ/h)' : d.iloc[target_id][target],
        }

    table = pd.DataFrame({
        '主たる居室' : get_value('主たる居室'),
        'その他の居室' : get_value('その他の居室'),
        '全居室' : get_value('全居室'),
    }).T
    table['index'] = table['index'].astype('int64')
    return table

# 居室連続
tables_cnt = {}
for (r,d) in datas_cnt.items():
    tables_cnt[r] = get_id_values(d)
tables_int = {}
# 居室間歇
for (r,d) in datas_int.items():
    tables_int[r] = get_id_values(d)

### 確認

In [10]:
for (r,t) in tables_cnt.items():
    print('地域:'+str(r))
    display(t)

地域:1


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1205,-24.5,0.5,14944.32
その他の居室,582,-23.0,0.5,11510.83
全居室,1205,-24.5,0.5,26297.48


地域:2


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1205,-19.4,0.6,12948.16
その他の居室,414,-16.0,0.9,9842.83
全居室,1205,-19.4,0.6,22731.97


地域:3


...

,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1061,-8.1,1.6,13581.44
その他の居室,534,-7.9,1.9,10983.59
全居室,1061,-8.1,1.6,24317.55


地域:6


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1565,-4.0,2.5,11426.96
その他の居室,8598,-2.1,2.5,9228.82
全居室,1565,-4.0,2.5,20380.52


地域:7


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1373,-5.1,1.6,12117.56
その他の居室,1374,-5.8,1.5,9883.79
全居室,1373,-5.1,1.6,21812.51


In [11]:
for (r,t) in tables_int.items():
    print('地域:'+str(r))
    display(t)

地域:1


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1205,-24.5,0.5,45334.00
その他の居室,152,-18.0,0.9,18704.89
全居室,343,-18.1,0.9,59152.85


地域:2


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1205,-19.4,0.6,38176.89
その他の居室,1328,-8.3,1.4,15169.71
全居室,1183,-16.2,0.7,46342.50


地域:3


...

,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,533,-7.1,1.8,38222.42
その他の居室,1328,-0.1,3.6,12476.72
全居室,487,-3.8,2.2,42207.40


地域:6


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1301,-2.0,2.0,30968.24
その他の居室,1328,0.0,2.6,12927.60
全居室,1327,-2.2,2.3,38916.69


地域:7


,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h)
主たる居室,1373,-5.1,1.6,34515.98
その他の居室,1352,1.0,2.2,11312.96
全居室,847,-2.4,3.1,37710.68


## 単位換算（kJ/h から W）値を追加する

In [12]:
for t in tables_cnt.values():
    t['負荷(W)'] = t['負荷(kJ/h)'] / 3.6
for t in tables_int.values():
    t['負荷(W)'] = t['負荷(kJ/h)'] / 3.6

### 確認（居室連続・1地域のみ）

In [13]:
tables_cnt['1']

,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h),負荷(W)
主たる居室,1205,-24.5,0.5,14944.32,4151.200000
その他の居室,582,-23.0,0.5,11510.83,3197.452778
全居室,1205,-24.5,0.5,26297.48,7304.855556


## 面積あたりに変換する

主たる居室・その他の居室・全居室の面積は以下のとおり。

| 場所 | 面積(m2) |
| --- | --- |
| 主たる居室 | 29.81 |
| その他の居室 | 51.34 |
| 全居室 | 81.15 |

In [14]:
area = pd.Series([29.81, 51.34, 81.15], index=['主たる居室', 'その他の居室', '全居室'])

面積で割ったあとに、少数第二位までの値に四捨五入して丸める

In [15]:
for (r,t) in tables_cnt.items():
    t['面積あたりの負荷(W/m2)'] = round(t['負荷(W)'] / area, 2)
for (r,t) in tables_int.items():
    t['面積あたりの負荷(W/m2)'] = round(t['負荷(W)'] / area, 2)

### 確認（居室連続・1地域のみ）

In [16]:
tables_cnt['1']

,index,外気温度(℃),外気湿度(g/kgDA),負荷(kJ/h),負荷(W),面積あたりの負荷(W/m2)
主たる居室,1205,-24.5,0.5,14944.32,4151.200000,139.26
その他の居室,582,-23.0,0.5,11510.83,3197.452778,62.28
全居室,1205,-24.5,0.5,26297.48,7304.855556,90.02


## 「連続運転時における単位面積あたりの最大暖房負荷を表す係数」の確認

$\alpha$ : 連続運転時における単位面積あたりの最大暖房負荷を表す係数, W m<sup>-2</sup>  

In [17]:
table_alpha = pd.DataFrame()
for (r,t) in tables_cnt.items():
    table_alpha[r+'地域'] = t['面積あたりの負荷(W/m2)']

In [18]:
table_alpha

,1地域,2地域,3地域,4地域,5地域,6地域,7地域
主たる居室,139.26,120.65,111.32,118.98,126.56,106.48,112.91
その他の居室,62.28,53.26,53.81,55.41,59.43,49.93,53.48
全居室,90.02,77.81,73.86,77.74,83.24,69.76,74.66


## その際使用した外気温湿度

In [19]:
table_temp = pd.DataFrame()
for (r,t)  in tables_cnt.items():
    table_temp[r+'地域'] = t['外気温度(℃)']

In [20]:
table_hum = pd.DataFrame()
for (r,t)  in tables_cnt.items():
    table_hum[r+'地域'] = t['外気湿度(g/kgDA)']

In [21]:
print('単位：℃')
display(table_temp)

単位：℃


,1地域,2地域,3地域,4地域,5地域,6地域,7地域
主たる居室,-24.5,-19.4,-13.0,-8.9,-8.1,-4.0,-5.1
その他の居室,-23.0,-16.0,-12.6,-9.4,-7.9,-2.1,-5.8
全居室,-24.5,-19.4,-13.0,-8.9,-8.1,-4.0,-5.1


In [22]:
print('単位:g/kgDA')
display(table_hum)

単位:g/kgDA


,1地域,2地域,3地域,4地域,5地域,6地域,7地域
主たる居室,0.5,0.6,1.1,1.7,1.6,2.5,1.6
その他の居室,0.5,0.9,1.1,1.5,1.9,2.5,1.5
全居室,0.5,0.6,1.1,1.7,1.6,2.5,1.6


## 間歇運転能力補正係数

連続と間歇それぞれで地域ごとにまとめた負荷テーブルをだす

In [23]:
table_cnt_load = pd.DataFrame()
for (r,t) in tables_cnt.items():
    table_cnt_load[r+'地域'] = t['負荷(W)']

In [24]:
table_int_load = pd.DataFrame()
for (r,t) in tables_int.items():
    table_int_load[r+'地域'] = t['負荷(W)']

In [25]:
table_cnt_load

,1地域,2地域,3地域,4地域,5地域,6地域,7地域
主たる居室,4151.200000,3596.711111,3318.580556,3546.869444,3772.622222,3174.155556,3365.988889
その他の居室,3197.452778,2734.119444,2762.661111,2844.775000,3050.997222,2563.561111,2745.497222
全居室,7304.855556,6314.436111,5994.038889,6308.886111,6754.875000,5661.255556,6059.030556


In [26]:
table_int_load

,1地域,2地域,3地域,4地域,5地域,6地域,7地域
主たる居室,12592.777778,10604.691667,9615.708333,10067.086111,10617.338889,8602.288889,9587.772222
その他の居室,5195.802778,4213.808333,3801.683333,3816.169444,3465.755556,3591.000000,3142.488889
全居室,16431.347222,12872.916667,11423.344444,12227.761111,11724.277778,10810.191667,10475.188889


運転方法や地域ごとに、連続運転の負荷に対する間歇運転の負荷の比をとる

In [27]:
f_cl_each = table_int_load/table_cnt_load

In [28]:
f_cl_each

,1地域,2地域,3地域,4地域,5地域,6地域,7地域
主たる居室,3.033527,2.948441,2.897537,2.838302,2.814313,2.710103,2.848427
その他の居室,1.624982,1.541194,1.376095,1.341466,1.135942,1.400786,1.144597
全居室,2.249373,2.038649,1.905784,1.938181,1.735676,1.909504,1.728856


運転方法ごとに、全地域を対象として最大値をとったものが間歇運転能力補正係数である。（少数第二位までで四捨五入）

In [29]:
f_cl = f_cl_each.max(axis=1).round(2)

In [30]:
f_cl

主たる居室     3.03
その他の居室    1.62
全居室       2.25
dtype: float64